#**Configurations**
This is where all configs will be set before the script makes an API call [via openmeteo] to collect weather data.

In [88]:
locations = {\
            #  "22.28185990473875, 114.18903700821926" : "Victoria Park",\
             "22.27054210988102, 114.00032030347596" : "Mui Wo",\
             "22.540797536909047, 114.43437970480949" : "Tung Ping Chau",\
             "22.468108498629288, 114.36264537675595" : "Tap Mun",\
            #  "22.23398579735171, 114.15930196830752" : "Ap Lei Chau Sandbar"\
             }

MAX_WIND_kph     =37.8    # [DJI Mini SE] wind resistance of up to 37.8 km/h
MAX_PRECIP_mm    =0
MAX_CLOUDS_pcnt  =50

#**Processing of raw data**

In [89]:
from urllib.request import urlopen, Request
import json
import pandas as pd
import datetime
import pytz

In [90]:
def get_df(latlong, name, maxwind_kph=37.8, maxprecip_mm=0, maxclouds_pcnt=100):
  # Construct URL
  gps_loc = latlong.split(",")
  latitude = str(round(float(gps_loc[0].strip()),4))
  longitude = str(round(float(gps_loc[1].strip()),4))
  url = 'https://api.open-meteo.com/v1/forecast?latitude=' + latitude + '&longitude=' + longitude +'&hourly=precipitation,cloudcover,windgusts_10m&timezone=auto'
  print(url)
  print("Weather Forecast: {} \nGPS: {}".format(name, latlong))

  # Retrieve data
  request = Request(url)
  response = urlopen(request)
  forecast = response.read()
  data = json.loads(forecast)
  df_json = pd.json_normalize(data)

  list_gust = df_json["hourly.windgusts_10m"][0]
  list_time = df_json["hourly.time"][0]
  list_precip = df_json["hourly.precipitation"][0]
  list_cloud = df_json["hourly.cloudcover"][0]
  assert len(list_gust) == len(list_time) and len(list_time) == len(list_precip) and len(list_precip) == len(list_cloud)

  # Init. custom DF
  timezone = df_json['timezone'][0]
  COL_TIME = "Datetime (" + timezone + ")"
  COL_GUST = "Gust (" + df_json['hourly_units.windgusts_10m'][0] + ")"
  COL_PRECIP = "Precip. (" + df_json['hourly_units.precipitation'][0] + ")"
  COL_CLOUD = "Cloud Cover (" + df_json['hourly_units.cloudcover'][0] + ")"
  df_custom = pd.DataFrame({COL_TIME : list_time, \
                            COL_GUST : list_gust, \
                            COL_PRECIP : list_precip, \
                            COL_CLOUD : list_cloud})

  # Constructing datetime filter
  tz = df_json["timezone"][0]
  datetimenow = datetime.datetime.now(pytz.timezone(tz))
  datefilter = []
  dayofweek = []
  for s in list_time:
    substr1 = s.split('-')
    yyyy, mm = int(substr1[0]), int(substr1[1])
    substr2 = substr1[2].split('T')
    dd, hh = int(substr2[0]), int(substr2[1].split(':')[0])
    is_daytime = hh < 18 and hh > 6
    datefilter.append(dd >= datetimenow.day and mm >= datetimenow.month and yyyy >= datetimenow.year and is_daytime)
    dayofweek.append(datetime.date(yyyy,mm,dd).strftime('%A'))

  df_custom["Day"] = dayofweek
  df_custom["Date Filter"] = datefilter

  # Filter: Gust & Precip. & Datetime
  df_custom_filtered = df_custom[(df_custom[COL_GUST] <= maxwind_kph)\
                                 & (df_custom[COL_PRECIP] <= maxprecip_mm)\
                                 & (df_custom[COL_CLOUD] <= maxclouds_pcnt)\
                                 & (df_custom["Date Filter"])\
                                 ].drop(columns=["Date Filter", COL_PRECIP]) 
  
  # Verbose
  print("Report generated on: " + datetimenow.strftime("%A %d. %B %Y"))
  print('{}\t{}\t{}\t{}'.format(COL_TIME, df_custom_filtered.columns[1], df_custom_filtered.columns[2], df_custom_filtered.columns[3]))
  for index, row in df_custom_filtered.iterrows():
    for e in row:
      print(e, end="\t\t")
    print('')
  print('\n')

  return df_custom_filtered 
  

# **Output**

In [91]:
# Reporting:
for key in locations:
  name = locations[key].split(";")  # TODO: parse optional arguments, ie. suffix -> ;67 -> filter out only Saturday and Sunday rows.
  df = get_df(key, name[0], MAX_WIND_kph, MAX_PRECIP_mm, MAX_CLOUDS_pcnt)


https://api.open-meteo.com/v1/forecast?latitude=22.2705&longitude=114.0003&hourly=precipitation,cloudcover,windgusts_10m&timezone=auto
Weather Forecast: Mui Wo 
GPS: 22.27054210988102, 114.00032030347596
Report generated on: Tuesday 25. October 2022
Datetime (Asia/Hong_Kong)	Gust (km/h)	Cloud Cover (%)	Day
2022-10-26T07:00		37.1		20		Wednesday		
2022-10-26T17:00		37.1		5		Wednesday		
2022-10-27T07:00		30.2		33		Thursday		
2022-10-27T08:00		34.2		31		Thursday		
2022-10-27T16:00		37.8		0		Thursday		
2022-10-27T17:00		34.6		0		Thursday		
2022-10-28T07:00		25.6		38		Friday		
2022-10-28T08:00		19.8		27		Friday		
2022-10-28T09:00		19.8		8		Friday		
2022-10-28T10:00		18.7		0		Friday		
2022-10-28T11:00		20.9		10		Friday		
2022-10-28T12:00		25.2		22		Friday		
2022-10-28T13:00		28.1		3		Friday		
2022-10-28T14:00		27.7		6		Friday		
2022-10-28T15:00		26.3		15		Friday		
2022-10-28T16:00		24.5		23		Friday		
2022-10-28T17:00		23.0		32		Friday		
2022-10-29T07:00		20.5		3		Saturday		
2022-10-29T08:00		